In [3]:
import os
import numpy
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
import astropy.io.fits as fits
from astropy.coordinates import SkyCoord, get_sun
from astropy.coordinates import Angle, Latitude, Longitude
from astropy.coordinates import ICRS, Galactic, FK4, FK5
import astropy.units as u
from astropy.time import Time
from astropy.table import Table

%matplotlib inline

# exec(open('skyplot_v5.py').read())

version = '8'

outdir = './figures/new'
if not os.path.exists(outdir):
    os.makedirs(outdir)


In [6]:
######################################################
#fitstable = '/Users/knyland/Desktop/VLASS/REALFAST/VLASS_realfast_IPS_candidates_elongation_v2.fits'
#fitstable = '/Users/knyland/Desktop/VLASS/REALFAST/VLASS_detected_realfast_IPS_candidates_elongation_v2.fits'
fitstable = 'IPS_candidates_Feb2022.fits'
tab = Table.read(fitstable)
ra = tab['RA']
dec = tab['Dec']
coords = SkyCoord(tab['RA'], tab['Dec'], unit=u.deg)
elongation = tab['Solar elon']
#epoch = numpy.array(tab['epoch']).astype('float')
sources = tab['CandId']
sources_list = list(sources)

names = []; mjd = []; epoch = []; time = []
for i in range(0,len(sources)):
    time_mjd = sources_list[i].split('.')[4]+'.'+sources_list[i].split('.')[5]
    mjd.append(time_mjd)
    epoch.append(sources[i].split('VLASS')[1].split('.sb')[0])
    time_fits = Time(time_mjd,format='mjd').fits
    time.append(time_fits)
    ra_name = "%02d" % int(coords[i].ra.hms[0]) + "%02d" % int(coords[i].ra.hms[1])
    dec_name = "%02d" % int(coords[i].dec.signed_dms[1]) + "%02d" % int(coords[i].dec.signed_dms[1])
    sign = "%01d" % int(coords[i].dec.signed_dms[0])
    if sign == '-1':
        names.append('J' + ra_name + '-' + dec_name)
    elif sign == '1':
        names.append('J' + ra_name + '+' + dec_name)

#Time('51544.000000000000001', format='mjd')
times = Time(mjd,format='mjd')
suns = get_sun(times)
tab['sun_ra'] = suns.ra
tab['sun_dec'] = suns.dec
elongation_check = suns.separation(coords)


tab['elongation'] = suns.separation(coords)
tab['epoch'] = epoch
tab['time'] = time
tab['names'] = names
tab.write("IPS_candidates_Feb2022_check.fits")




 59537.19167758 59537.19167758 59554.31438322 59554.31438322
 59554.31438322 59593.55282584 59594.6410308  59594.6410308
 59594.6410308  59594.6410308  59594.6410308  59594.6410308
 59594.6410308  59594.6410308  59594.6410308  59594.6410308
 59593.55282584 59593.55282584 59594.6410308  59594.6410308
 59594.6410308  59594.6410308  59593.55282584 59594.6410308
 59594.6410308  59594.6410308  59594.6410308  59594.6410308
 59594.6410308  59594.6410308  59594.6410308  59594.6410308
 59594.6410308  59594.6410308  59594.6410308  59594.6410308
 59594.6410308  59594.6410308  59594.6410308  59594.6410308
 59594.6410308  59594.6410308  59594.6410308  59594.6410308
 59594.6410308  59594.6410308  59593.55282584 59593.55282584
 59610.48876689 59610.48876689 59610.48876689 59610.48876689
 59610.48876689 59593.55282584 59611.81469002 59610.48876689
 59610.48876689 59610.48876689 59611.81469002 59610.48876689
 59609.81940617 59593.55282584 59611.81469002 59614.65410378
 59614.65410378 59614.65410378 596

In [7]:
#####################################################
#plt.ioff()
plt.ion()
plt.rc('font',family='sans-serif')
plt.rc('font', serif='Helvetica')
plt.rcParams['ytick.major.pad']='5'
plt.rcParams['xtick.major.pad']='5'
plt.rcParams['mathtext.fontset']='custom'
plt.rcParams['mathtext.rm']='Bitstream Vera Sans'
plt.rcParams['mathtext.it']='Bitstream Vera Sans:italic'
plt.rcParams['mathtext.bf']='Bitstream Vera Sans:bold'
sns.set_style({"xtick.direction": "in","ytick.direction": "in"})
pal = sns.set_palette("muted")
sns.set_style("whitegrid", {'axes.linewidth': 1.5, 'axes.grid': True,'axes.edgecolor': '.15','grid.color': '.8', 'xtick.color': '.15', 'ytick.color': '.15','xtick.direction': u'in','ytick.direction': u'in'})
#cmap = ListedColormap(sns.color_palette(n_colors=10))
#colors = cmap.colors
hist_kwsa = dict(histtype='stepfilled',linewidth=3,alpha=0.05)
hist_kwsb = dict(histtype='step',linewidth=3,alpha=1.0)

######################################################
fig = plt.figure(figsize=(10,5))

ra_plot = Angle(ra*u.deg).wrap_at(180*u.degree).radian
dec_plot = Angle(dec*u.deg).radian
#elongation_plot = (Angle(elongation*u.deg).radian).tolist()
ra_plot_epoch2 = ra_plot[numpy.where(epoch==2.2)]
dec_plot_epoch2 = dec_plot[numpy.where(epoch==2.2)]
elongation_plot_epoch2 = elongation[numpy.where(epoch==2.2)]

ax = fig.add_subplot(111, projection='mollweide')
#plot=ax.scatter(ra_plot, dec_plot, c='mediumslateblue', alpha=0.5, lw=0, marker='o',s=15, label='IPS Candidates')
plot=ax.scatter(ra_plot, dec_plot, c=numpy.abs(elongation), cmap='Oranges_r', edgecolors='k',alpha=0.75, lw=0.5, marker='o',s=30, label=None)
#plot=ax.scatter(ra_plot_epoch2, dec_plot_epoch2, c=elongation_plot_epoch2, cmap='Oranges_r', edgecolors='k',alpha=0.75, lw=0.5, marker='o',s=20, label=None)
ax.tick_params(labelsize=0)
ax.set_xticklabels(['14h', '16h','18h', '20h', '22h', '0h','2h','4h','6h','8h','10h'],zorder=2)
#ax.axes.xaxis.set_visible(False)
#ax.axes.yaxis.set_visible(False)

# Add galactic plane and galactic center
lgal=numpy.arange(360)
bgal=[0.]*360
bgalp10=[10.]*360
bgalm10=[-10.]*360
gplane_arr = numpy.array(list(zip(lgal,bgal)))
c_gp = SkyCoord(gplane_arr * u.deg,frame='galactic').icrs
gplanep10_arr=numpy.array(list(zip(lgal,bgalp10)))
c_gpp10 = SkyCoord(gplanep10_arr*u.deg,frame='galactic').icrs
gplanem10 = numpy.array(list(zip(lgal,bgalm10)))
c_gpm10 = SkyCoord(gplanem10*u.deg,frame='galactic').icrs
#gcen=SkyCoord(0.0*u.deg,0.0*u.deg,frame='galactic').icrs
#ax.plot(c_gpp10.ra.wrap_at(180*u.deg).rad, c_gpp10.dec.rad, color='k', marker='.', alpha=0.3, markersize=3, linewidth=0) # galactic plane + 10 deg
#ax.plot(c_gpm10.ra.wrap_at(180*u.deg).rad, c_gpm10.dec.rad, color='k', marker='.', alpha=0.3, markersize=3, linewidth=0) # galactic plane - 10 deg
#ax.plot(c_gp.ra.wrap_at(180*u.deg).rad, c_gp.dec.rad, color='k', marker='.', alpha=0.3, markersize=3, linewidth=0) # galactic plane + 10 deg
ax.scatter(c_gp.ra.wrap_at(180*u.deg).rad[::4], c_gp.dec.rad[::4], c='k', alpha=0.3, lw=0, marker='D',s=10, label='Galactic Plane')
#ax.plot(c_gp.ra.wrap_at(180*u.deg).rad, c_gp.dec.rad, color='mediumslateblue', alpha=0.5, lw=3, marker=None, label='Galactic Plane')

# Add ecliptic
lon_ecl = numpy.linspace(0, 360, 1000)
lat_ecl = numpy.zeros(1000)

# Transform ecliptic coordinates to galactic
ecl = SkyCoord(lon_ecl, lat_ecl, unit=u.deg, frame='barycentricmeanecliptic')
ecl_icrs = ecl.transform_to('icrs')
#ra_ecl_icrs, dec_ecl_icrs = -ecl_icrs.ra.wrap_at('180d').radian, ecl_icrs.dec.radian # WRONG SIGN
ra_ecl_icrs, dec_ecl_icrs = ecl_icrs.ra.wrap_at('180d').radian, ecl_icrs.dec.radian
ax.scatter(ra_ecl_icrs[::10], dec_ecl_icrs[::10], c='k', alpha=0.75, lw=0, marker='s',s=10, label='Ecliptic Plane')

# Add labels, etc. and save
plt.xlabel('RA',fontsize=0, fontweight='normal',labelpad=10)
plt.ylabel('DEC',fontsize=20, fontweight='normal',labelpad=5)
plt.tick_params(which='major', axis='both',length=0, labelsize=10)
plt.tick_params(which='minor', axis='both',length=0, labelsize=10)
cbar = plt.colorbar(plot, orientation='horizontal',fraction=0.075,pad=0.05)
#cbar = plt.colorbar(plot, orientation='vertical',fraction=0.075,pad=0.05)
cbar.ax.tick_params(direction='in', length=0, labelsize=16)
cbar.set_label('Solar Elongation (deg)', rotation=0, fontsize=22)
leg=plt.legend(loc='lower center', fontsize=13, frameon=True)
#leg=plt.legend(fontsize=13, frameon=True)
plt.suptitle('IPS Candidates',fontsize=28, y=0.95)
#plt.suptitle('IPS Candidates - VLASS Detected',fontsize=28, y=0.95)
plt.tight_layout()
plt.savefig(outdir + '/' + 'IPS_candididates_all_skyplot_v' + version + '.png', dpi=300)
#plt.savefig(outdir + '/' + 'IPS_candididates_all_skyplot_epoch2_v' + version + '.png', dpi=300)
#plt.savefig(outdir + '/' + 'IPS_candididates_VLASS_det_skyplot_v' + version + '.png', dpi=300)
#plt.savefig(outdir + '/' + 'IPS_candididates_VLASS_det_skyplot_epoch2_v' + version + '.png', dpi=300)


C:\Users\jray\AppData\Local\Temp\ipykernel_15364\3350813002.py:26: DeprecationWarning: Calling nonzero on 0d arrays is deprecated, as it behaves surprisingly. Use `atleast_1d(cond).nonzero()` if the old behavior was intended. If the context of this warning is of the form `arr[nonzero(cond)]`, just use `arr[cond]`.
  ra_plot_epoch2 = ra_plot[numpy.where(epoch==2.2)]
C:\Users\jray\AppData\Local\Temp\ipykernel_15364\3350813002.py:27: DeprecationWarning: Calling nonzero on 0d arrays is deprecated, as it behaves surprisingly. Use `atleast_1d(cond).nonzero()` if the old behavior was intended. If the context of this warning is of the form `arr[nonzero(cond)]`, just use `arr[cond]`.
  dec_plot_epoch2 = dec_plot[numpy.where(epoch==2.2)]
C:\Users\jray\AppData\Local\Temp\ipykernel_15364\3350813002.py:28: DeprecationWarning: Calling nonzero on 0d arrays is deprecated, as it behaves surprisingly. Use `atleast_1d(cond).nonzero()` if the old behavior was intended. If the context of this warning is of